## Setup and Import Libraries

In [1]:
import os
import requests
import datetime
import wikipedia
from langchain.memory import ConversationBufferMemory
from langchain_core.tools import tool
from langchain_openai.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from langchain_core.utils.function_calling import format_tool_to_openai_function
from langchain_core.agents import AgentFinish
from langchain_core.runnables.passthrough import RunnablePassthrough
from langchain.agents.output_parsers.openai_functions import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad.openai_functions import format_to_openai_functions
from langchain.agents.agent import AgentExecutor
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [4]:
model = ChatOpenAI()

## Tools

In [5]:
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [6]:
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [7]:
tools = [get_current_temperature, search_wikipedia]

In [9]:
functions = [format_tool_to_openai_function(f) for f in tools]

In [10]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
])

In [11]:
model = model.bind(functions=functions)

In [12]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [13]:
response = chain.invoke({"input": "what is the weather is sf?"})

In [15]:
response.tool

'get_current_temperature'

In [16]:
response.tool_input

{'latitude': 37.7749, 'longitude': -122.4194}

In [17]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [18]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser()

In [19]:
response = chain.invoke({
    "input": "what is the weather is sf?",
    "agent_scratchpad": []
})

In [20]:
response.tool

'get_current_temperature'

In [22]:
observation = get_current_temperature(response.tool_input)

In [23]:
observation

'The current temperature is 13.6°C'

In [24]:
type(response)

langchain_core.agents.AgentActionMessageLog

In [25]:
response.message_log

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BuEJxQcnBtnaRP934Kaw2WhP44ygX', 'finish_reason': 'function_call', 'logprobs': None}, id='run--336d0da7-c6d9-4838-85c6-5439cb7bbb34-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})]

In [26]:
format_to_openai_functions([(response, observation), ])

[AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{"latitude":37.7749,"longitude":-122.4194}', 'name': 'get_current_temperature'}, 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 112, 'total_tokens': 137, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-BuEJxQcnBtnaRP934Kaw2WhP44ygX', 'finish_reason': 'function_call', 'logprobs': None}, id='run--336d0da7-c6d9-4838-85c6-5439cb7bbb34-0', usage_metadata={'input_tokens': 112, 'output_tokens': 25, 'total_tokens': 137, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}),
 FunctionMessage(content='The current temperature is 13.6°C', additional_kwargs={}, response_metadata={}, name='ge

In [27]:
response2 = chain.invoke({
    "input": "what is the weather is sf?", 
    "agent_scratchpad": format_to_openai_functions([(response, observation)])
})

In [28]:
response2

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 13.6°C.'}, log='The current temperature in San Francisco is 13.6°C.')

### Run Agent

In [29]:
def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = chain.invoke({
            "input": user_input, 
            "agent_scratchpad": format_to_openai_functions(intermediate_steps)
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

In [30]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | chain

In [31]:
run_agent("what is the weather is sf?")

AgentFinish(return_values={'output': 'The current temperature in San Francisco is 13.6°C.'}, log='The current temperature in San Francisco is 13.6°C.')

In [32]:
run_agent("what is langchain?")

AgentFinish(return_values={'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It overlaps with the use cases of language models in general, including document analysis and summarization, chatbots, and code analysis.'}, log='LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It overlaps with the use cases of language models in general, including document analysis and summarization, chatbots, and code analysis.')

In [33]:
run_agent("hi!")

AgentFinish(return_values={'output': 'Well, hello there! How can I assist you today?'}, log='Well, hello there! How can I assist you today?')

### Agent Executor

In [34]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True)

In [35]:
agent_executor.invoke({"input": "what is langchain?"})



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'langchain'}`


Page: LangChain
Summary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.



Page: Model Context Protocol
Summary: The Model Context Protocol (MCP) is an open standard, open-source framework introduced by Anthropic in November 2024 to standardize the way artificial intelligence (AI) systems like large language models (LLMs) integrate and share data with external tools, systems, and data sources. MCP provides a universal interface for reading files, executing functions, and handling contextual prompts. Following its announcement, the protocol was adopted by major AI providers, including OpenAI and Google DeepMind.

Pag

{'input': 'what is langchain?',
 'output': 'LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. It is used for various purposes such as document analysis and summarization, chatbots, and code analysis.'}

In [36]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [37]:
agent_executor.invoke({"input": "what is my name"})



> Entering new AgentExecutor chain...
I'm sorry, I don't have that information. How can I assist you today?

> Finished chain.


{'input': 'what is my name',
 'output': "I'm sorry, I don't have that information. How can I assist you today?"}

## Chat History

In [38]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are helpful but sassy assistant"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad")
])

In [39]:
agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

In [41]:
memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")

In [42]:
agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [43]:
agent_executor.invoke({"input": "whats the weather in sf?"})



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 37.7749, 'longitude': -122.4194}`


The current temperature is 13.6°CThe current temperature in San Francisco is 13.6°C.

> Finished chain.


{'input': 'whats the weather in sf?',
 'chat_history': [HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in San Francisco is 13.6°C.', additional_kwargs={}, response_metadata={})],
 'output': 'The current temperature in San Francisco is 13.6°C.'}

In [44]:
agent_executor.invoke({"input": "my name is bob"})



> Entering new AgentExecutor chain...
Nice to meet you, Bob! How can I assist you today?

> Finished chain.


{'input': 'my name is bob',
 'chat_history': [HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in San Francisco is 13.6°C.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'output': 'Nice to meet you, Bob! How can I assist you today?'}

In [45]:
agent_executor.invoke({"input": "whats my name"})



> Entering new AgentExecutor chain...
You just told me your name is Bob. So, your name is Bob.

> Finished chain.


{'input': 'whats my name',
 'chat_history': [HumanMessage(content='whats the weather in sf?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current temperature in San Francisco is 13.6°C.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is bob', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Bob! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='whats my name', additional_kwargs={}, response_metadata={}),
  AIMessage(content='You just told me your name is Bob. So, your name is Bob.', additional_kwargs={}, response_metadata={})],
 'output': 'You just told me your name is Bob. So, your name is Bob.'}

## Create Chatbot

In [46]:
@tool
def create_your_own(query: str) -> str:
    """This function can do whatever you would like once you fill it in """
    print(type(query))
    return query[::-1]

In [47]:
tools = [get_current_temperature, search_wikipedia, create_your_own]

In [50]:
import panel as pn
pn.extension()
import param

class cbfs(param.Parameterized):
    
    def __init__(self, tools, **params):
        super(cbfs, self).__init__( **params)
        self.panels = []
        self.functions = [format_tool_to_openai_function(f) for f in tools]
        self.model = ChatOpenAI(temperature=0).bind(functions=self.functions)
        self.memory = ConversationBufferMemory(return_messages=True,memory_key="chat_history")
        self.prompt = ChatPromptTemplate.from_messages([
            ("system", "You are helpful but sassy assistant"),
            MessagesPlaceholder(variable_name="chat_history"),
            ("user", "{input}"),
            MessagesPlaceholder(variable_name="agent_scratchpad")
        ])
        self.chain = RunnablePassthrough.assign(
            agent_scratchpad = lambda x: format_to_openai_functions(x["intermediate_steps"])
        ) | self.prompt | self.model | OpenAIFunctionsAgentOutputParser()
        self.qa = AgentExecutor(agent=self.chain, tools=tools, verbose=False, memory=self.memory)
    
    def convchain(self, query):
        if not query:
            return
        inp.value = ''
        result = self.qa.invoke({"input": query})
        self.answer = result['output'] 
        self.panels.extend([
            pn.Row('User:', pn.pane.Markdown(query, width=450)),
            pn.Row('ChatBot:', pn.pane.Markdown(self.answer, width=450, styles={'background-color': '#F6F6F6'}))
        ])
        return pn.WidgetBox(*self.panels, scroll=True)


    def clr_history(self,count=0):
        self.chat_history = []
        return 

In [54]:
cb = cbfs(tools)

inp = pn.widgets.TextInput( placeholder='Enter text here…')

conversation = pn.bind(cb.convchain, inp) 

tab1 = pn.Column(
    pn.Row(inp),
    pn.layout.Divider(),
    pn.panel(conversation,  loading_indicator=True, height=400),
    pn.layout.Divider(),
)

dashboard = pn.Column(
    pn.Row(pn.pane.Markdown('# Conversational Bot')),
    pn.Tabs(('Conversation', tab1))
)

# dashboard